In [1]:
import numpy as np
import mxnet as mx
from collections import namedtuple, Counter
from unidecode import unidecode
from itertools import groupby
from mxnet.io import DataIter
from random import shuffle

import deepdish as dd

import operator
import pickle
import re
import warnings

/usr/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-03-30 17:44:52.398851. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [2]:
# Get rid of annoying Python deprecation warnings from built-in JSON encoder
warnings.filterwarnings("ignore", category=DeprecationWarning)   

In [3]:
# Decode text as UTF-8
# Remove diacritical signs and convert to Latin alphabet
# Separate punctuation as separate "words"
def tokenize_text(fname, vocab=None, invalid_label=0, start_label=1, sep_punctuation=True):
    lines = unidecode(open(fname).read().decode('utf-8')).split('\n')
    lines = [x for x in lines if x]
    lines = map(lambda x: re.findall(r"\w+|[^\w\s]", x, re.UNICODE), lines)    
    sentences, vocab = mx.rnn.encode_sentences(lines, vocab=vocab, invalid_label=invalid_label, start_label=start_label)
    return sentences, vocab

Dataset = namedtuple(
    'Dataset', 
    ['src_sent', 'src_vocab', 'inv_src_vocab', 'targ_sent', 'targ_vocab', 'inv_targ_vocab'])

def invert_dict(d):
    return {v: k for k, v in d.iteritems()}


def get_data(src_path, targ_path, start_label=1, invalid_label=0, pad_symbol='<PAD>'):
    src_sent, src_vocab = tokenize_text(src_path, start_label=start_label,
                                invalid_label=invalid_label)
    
    src_vocab[pad_symbol] = invalid_label
    inv_src_vocab = invert_dict(src_vocab)

    targ_sent, targ_vocab = tokenize_text(targ_path, start_label=start_label, #new_start+1,
                                          invalid_label=invalid_label)
    
    targ_vocab[pad_symbol] = invalid_label
    inv_targ_vocab = invert_dict(targ_vocab)
    
    return Dataset(
        src_sent=src_sent, src_vocab=src_vocab, inv_src_vocab=inv_src_vocab,
        targ_sent=targ_sent, targ_vocab=targ_vocab, inv_targ_vocab=inv_targ_vocab)

In [4]:
def persist_dataset(dataset, path):
    with open(path, 'wb+') as fileobj:
        pickle.dump(dataset, fileobj)
        
def load_dataset(path):
    with open(path, 'rb') as fileobj:
        return pickle.load(fileobj)

In [5]:
dataset = \
    get_data(
        src_path='./data/europarl-v7.es-en.en_small',
        targ_path='./data/europarl-v7.es-en.es_small',
        start_label=1,
        invalid_label=0
    )

In [169]:
class TwoDBisect:
    def __init__(self, buckets):
        self.buckets = sorted(buckets, key=operator.itemgetter(0, 1))
        self.x, self.y = zip(*buckets)
        self.x, self.y = np.array(list(self.x)), np.array(list(self.y))

    def twod_bisect(self, source, target):    
        offset1 = np.searchsorted(self.x, len(source), side='left')
        offset2 = np.where(self.y[offset1:] >= len(target))[0]        
        return self.buckets[offset1 + offset2[0]] 

class Seq2SeqIterator:    
    
    def __init__(self, dataset, buckets=None, batch_size=32, max_sent_len=None):
        self.src_sent = dataset.src_sent
        self.targ_sent = dataset.targ_sent
        if buckets:
            z = zip(*buckets)
            self.max_sent_len = max(max(z[0]), max(z[1]))
        else:
            self.max_sent_len = max_sent_len
        if self.max_sent_len:
            self.src_sent, self.targ_sent = self.filter_long_sent(
                self.src_sent, self.targ_sent, self.max_sent_len) 
        self.src_vocab = dataset.src_vocab
        self.targ_vocab = dataset.targ_vocab
        self.inv_src_vocab = dataset.inv_src_vocab
        self.inv_targ_vocab = dataset.inv_targ_vocab
        # Can't filter smaller counts per bucket if those sentences still exist!
        self.buckets = buckets if buckets else self.gen_buckets(
            self.src_sent, self.targ_sent, filter_smaller_counts_than=1, max_sent_len=max_sent_len)
        self.bisect = TwoDBisect(self.buckets)
        self.max_sent_len = max_sent_len
        self.pad_id = self.src_vocab['<PAD>']
        # After bucketization, we should probably del self.src_sent and self.targ_sent
        # to free up memory.
        self.bucketed_data, self.bucket_idx_to_key = self.bucketize()
        self.bucket_key_to_idx = invert_dict(dict(enumerate(self.bucket_idx_to_key)))
        self.interbucket_idx = 0
        self.intrabucket_idx = 0
        self.bucket_iterator_indices = list(range(len(self.bucket_idx_to_key)))
    
    def bucketize(self):
        tuples = []
        ctr = 0
        for src, targ in zip(self.src_sent, self.targ_sent):
            len_tup = self.bisect.twod_bisect(src, targ)
            rev_src = src[::-1] 
            tuples.append((src, targ, len_tup))
            
        sorted_tuples = sorted(tuples, key=operator.itemgetter(2))
        grouped = groupby(sorted_tuples, lambda x: x[2])
        bucketed_data = [] 
        bucket_idx_to_key = []
        
        for group in grouped:
            
            # get src and targ sentences, ignore the last elem of the tuple 
            # (the grouping key of (src_len, targ_len))
            key, value = group[0], map(lambda x: x[:2], group[1])

            # create padded representation
            new_src = np.full((len(value), key[0]), self.pad_id, dtype=np.int32)
            new_targ = np.full((len(value), key[1]), self.pad_id, dtype=np.int32)
            
            for idx, example in enumerate(value):
                curr_src, curr_targ = example
                rev_src = curr_src[::-1]
                new_src[idx, :-(len(rev_src)+1):-1] = rev_src
                new_targ[idx, :len(curr_targ)] = curr_targ
                                
            bucketed_data.append((new_src, new_targ))
            bucket_idx_to_key.append(key)
        return bucketed_data, bucket_idx_to_key
    
    def current_bucket_key(self):
        return self.bucket_idx_to_key[self.interbucket_idx]
    
    def current_bucket_index(self):
        return self.bucket_iterator_indices[self.interbucket_idx]

    # shuffle the data within buckets, and reset iterator
    def reset(self):
        self.interbucket_idx = 0
        self.intrabucket_idx = 0        
        for idx in range(len(self.bucketed_data)):
            current = self.bucketed_data[idx]
            src, targ = current
            indices = np.array(range(len(src)))
            np.random.shuffle(indices)
            src = src[indices]
            targ = targ[indices]
            self.bucketed_data[idx] = (src, targ)
        shuffle(self.bucket_iterator_indices)
    

    # iterate over data
    def next(self):
        # raise StopIteration when done
        pass        

#     def next(self):
#         if self.curr_idx == len(self.idx):
#             raise StopIteration
#         i, j = self.idx[self.curr_idx]
#         self.curr_idx += 1

#         if self.major_axis == 1:
#             data = self.nddata[i][j:j+self.batch_size].T
#             label = self.ndlabel[i][j:j+self.batch_size].T
#         else:
#             data = self.nddata[i][j:j+self.batch_size]
#             label = self.ndlabel[i][j:j+self.batch_size]

#         return DataBatch([data], [label], pad=0,
#                          bucket_key=self.buckets[i],
#                          provide_data=[(self.data_name, data.shape)],
#                          provide_label=[(self.label_name, label.shape)])    
    
    @staticmethod 
    def filter_long_sent(src_sent, targ_sent, max_len):
        result = filter(lambda x: len(x[0]) <= max_len and len(x[1]) <= max_len, zip(src_sent, targ_sent))
        return zip(*result)

    @staticmethod
    def gen_buckets(src_sent, targ_sent, filter_smaller_counts_than=None, max_sent_len=60, min_sent_len=1):
        length_pairs = map(lambda x: (len(x[0]), len(x[1])), zip(src_sent, targ_sent))
        counts = list(Counter(length_pairs).items())
        c_sorted = sorted(counts, key=operator.itemgetter(0, 1))
        buckets = [i[0] for i in c_sorted if i[1] >= filter_smaller_counts_than and 
                   (max_sent_len is None or i[0][0] <= max_sent_len) and
                   (max_sent_len is None or i[0][1] <= max_sent_len) and
                   (min_sent_len is None or i[0][0] >= min_sent_len) and
                   (min_sent_len is None or i[0][1] >= min_sent_len)]
        return buckets

In [171]:
i1 = Seq2SeqIterator(dataset)

In [192]:
i1.reset()
i1.bucketed_data[10]

(array([[  561],
        [   36],
        [   36],
        [   36],
        [   36],
        [   36],
        [   36],
        [   36],
        [   36],
        [   36],
        [   36],
        [   70],
        [   36],
        [   36],
        [   36],
        [   36],
        [  561],
        [   36],
        [   36],
        [   36],
        [   36],
        [   36],
        [   36],
        [25151],
        [   36],
        [   36],
        [   36],
        [   36],
        [   36],
        [ 4688],
        [   36],
        [ 4688],
        [   36],
        [   36],
        [   36],
        [   36],
        [   36],
        [   36],
        [   36],
        [   36],
        [   36],
        [  561],
        [   36]], dtype=int32),
 array([[  811,  1281,    43,   689,   103,  1209,  8176,    66,    70,
           498,    30],
        [  822,    41,    42,   914,     4,   467,  4481,   650,    47,
         13241,    30],
        [  129,    31,   459,  2385,     8,  2204,  1098,   27

In [179]:
src_sent = dataset.src_sent
targ_sent = dataset.targ_sent

sent_len = lambda x: map(lambda y: len(y), x)
max_len = lambda x: max(sent_len(x))
min_len = lambda x: min(sent_len(x))

min_len = min(min(sent_len(src_sent)), min(sent_len(targ_sent)))
# max_len = max(max(sent_len(src_sent)), max(sent_len(targ_sent)))

# min_len = min
max_len = 65
increment = 5

all_pairs = [(i, j) for i in range(
        min_len,max_len+increment,increment
    ) for j in range(
        min_len,max_len+increment,increment
    )]

In [102]:
i2 = Seq2SeqIterator(dataset, buckets=all_pairs)

In [140]:
src, targ = i1.bucketed_data[10]

In [155]:
indices = np.array(range(len(src)))
np.random.shuffle(indices)
indices

array([20,  2,  9, 42, 22,  4,  6, 21,  1, 29, 19, 35, 32, 10, 16,  8, 26,
        7, 36, 28, 15, 38,  5, 31,  0, 41,  3, 25, 37, 30, 17, 23, 11, 18,
       12, 34, 39, 33, 14, 27, 24, 13, 40])

In [156]:
src, targ = i1.bucketed_data[10]

In [159]:
targ[indices]

array([[ 1448,   134,    11,   317,   242,   103,   362,  1730,   120,
          922,    30],
       [   97,    98,   134,  6128,  1139,    70,   279,  1026,   237,
         6129,   100],
       [   45,   246,   402,    11,    42,  4547,   404,  3802,     8,
          505,    30],
       [12555,     8,   648,     4,  7129,    70, 14681,    17,     8,
        34234,    30],
       [   98,  1318, 21781,     4,    86,  5346,  3232,   103,   319,
         1100,    30],
       [  574,   363,    42, 42033,  1161,    30,   938,    78,  3537,
          985,    30],
       [  963,    43,    52,  1315,   337,   922,     4,    70, 28781,
        30718,    30],
       [  938, 11531,   337,    19,    70,  5468,  8994,    19,    70,
         1069,    30],
       [  595,   214,  1315,    24,   332,   689,   103,  3542,     4,
         5967,    30],
       [  822,    41,    42,   914,     4,   467,  4481,   650,    47,
        13241,    30],
       [  963,  1513,    11,   108,  1129,    11,    86, 100